In [1]:
import xl
from utils import *
import tensorflow as tf
from sklearn.cross_validation import train_test_split
import time
import random
import os

/usr/local/lib/python3.6/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
trainset = sklearn.datasets.load_files(container_path = 'data', encoding = 'UTF-8')
trainset.data, trainset.target = separate_dataset(trainset,1.0)
print (trainset.target_names)
print (len(trainset.data))
print (len(trainset.target))

['negative', 'positive']
10662
10662


In [3]:
concat = ' '.join(trainset.data).split()
vocabulary_size = len(list(set(concat)))
data, count, dictionary, rev_dictionary = build_dataset(concat, vocabulary_size)
print('vocab from size: %d'%(vocabulary_size))
print('Most common words', count[4:10])
print('Sample data', data[:10], [rev_dictionary[i] for i in data[:10]])

vocab from size: 20332
Most common words [('film', 1453), ('movie', 1270), ('one', 727), ('like', 721), ('story', 477), ('much', 386)]
Sample data [28, 393, 18, 357, 4, 3117, 475, 849, 65, 4373] ['bad', 'journey', 'funny', 'touching', 'film', 'gorgeously', 'acted', 'british', 'cast', 'rival']


In [4]:
GO = dictionary['GO']
PAD = dictionary['PAD']
EOS = dictionary['EOS']
UNK = dictionary['UNK']

In [5]:
learning_rate = 1e-4
batch_size = 32
maxlen = 100
n_layer = 3
d_model = 256
d_embed = 256
n_head = 10
d_head = 50
d_inner = 512

In [6]:
class Model:
    def __init__(self):
        self.X = tf.placeholder(tf.int32, [None, maxlen])
        self.Y = tf.placeholder(tf.int32, [None])
        self.memory = tf.placeholder(tf.float32, [n_layer, None, maxlen, d_model])
        initializer = tf.initializers.random_normal(stddev = 0.1)
        logits, self.next_memory = xl.transformer(
            self.X,
            self.memory,
            len(dictionary),
            n_layer,
            d_model,
            d_embed,
            n_head,
            d_head,
            d_inner,
            initializer
        )
        
        self.logits = tf.layers.dense(logits[:, -1], 2)
        self.cost = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(
                logits = self.logits, labels = self.Y
            )
        )
        self.optimizer = tf.train.AdamOptimizer(learning_rate).minimize(
            self.cost
        )
        correct_pred = tf.equal(
            tf.argmax(self.logits, 1, output_type = tf.int32), self.Y
        )
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [7]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model()
sess.run(tf.global_variables_initializer())

In [8]:
vectors = str_idx(trainset.data,dictionary,maxlen)
train_X, test_X, train_Y, test_Y = train_test_split(vectors, trainset.target,test_size = 0.2)

In [9]:
from tqdm import tqdm
import time

EARLY_STOPPING, CURRENT_CHECKPOINT, CURRENT_ACC, EPOCH = 3, 0, 0, 0

while True:
    lasttime = time.time()
    if CURRENT_CHECKPOINT == EARLY_STOPPING:
        print('break epoch:%d\n' % (EPOCH))
        break

    train_acc, train_loss, test_acc, test_loss = 0, 0, 0, 0
    pbar = tqdm(
        range(0, len(train_X), batch_size), desc = 'train minibatch loop'
    )
    for i in pbar:
        batch_x = train_X[i : min(i + batch_size, train_X.shape[0])]
        batch_y = train_Y[i : min(i + batch_size, train_X.shape[0])]
        batch_memory = np.ones(([n_layer, batch_size, maxlen, d_model]))
        batch_x_expand = np.expand_dims(batch_x,axis = 1)
        acc, cost, _ = sess.run(
            [model.accuracy, model.cost, model.optimizer],
            feed_dict = {
                model.Y: batch_y,
                model.X: batch_x,
                model.memory: batch_memory
            },
        )
        assert not np.isnan(cost)
        train_loss += cost
        train_acc += acc
        pbar.set_postfix(cost = cost, accuracy = acc)
        
    pbar = tqdm(range(0, len(test_X), batch_size), desc = 'test minibatch loop')
    for i in pbar:
        batch_x = test_X[i : min(i + batch_size, test_X.shape[0])]
        batch_y = test_Y[i : min(i + batch_size, test_X.shape[0])]
        batch_memory = np.ones(([n_layer, batch_size, maxlen, d_model]))
        batch_x_expand = np.expand_dims(batch_x,axis = 1)
        acc, cost = sess.run(
            [model.accuracy, model.cost],
            feed_dict = {
                model.Y: batch_y,
                model.X: batch_x,
                model.memory: batch_memory
            },
        )
        test_loss += cost
        test_acc += acc
        pbar.set_postfix(cost = cost, accuracy = acc)

    train_loss /= len(train_X) / batch_size
    train_acc /= len(train_X) / batch_size
    test_loss /= len(test_X) / batch_size
    test_acc /= len(test_X) / batch_size

    if test_acc > CURRENT_ACC:
        print(
            'epoch: %d, pass acc: %f, current acc: %f'
            % (EPOCH, CURRENT_ACC, test_acc)
        )
        CURRENT_ACC = test_acc
        CURRENT_CHECKPOINT = 0
    else:
        CURRENT_CHECKPOINT += 1

    print('time taken:', time.time() - lasttime)
    print(
        'epoch: %d, training loss: %f, training acc: %f, valid loss: %f, valid acc: %f\n'
        % (EPOCH, train_loss, train_acc, test_loss, test_acc)
    )
    EPOCH += 1

train minibatch loop:   0%|          | 1/267 [00:00<00:34,  7.62it/s, accuracy=0.562, cost=0.691]

epoch: 0, pass acc: 0.000000, current acc: 0.518206
time taken: 39.01648306846619
epoch: 0, training loss: 0.713227, training acc: 0.501238, valid loss: 0.756064, valid acc: 0.518206



train minibatch loop:   0%|          | 1/267 [00:00<00:34,  7.63it/s, accuracy=0.688, cost=0.587]

epoch: 1, pass acc: 0.518206, current acc: 0.520997
time taken: 38.66717529296875
epoch: 1, training loss: 0.648455, training acc: 0.616699, valid loss: 0.748228, valid acc: 0.520997



train minibatch loop:   0%|          | 1/267 [00:00<00:34,  7.63it/s, accuracy=0.812, cost=0.46]

...

time taken: 38.664732933044434
epoch: 3, training loss: 0.387593, training acc: 0.817957, valid loss: 1.188958, valid acc: 0.515147



train minibatch loop:   0%|          | 1/267 [00:00<00:34,  7.63it/s, accuracy=0.938, cost=0.186]

time taken: 38.656598806381226
epoch: 4, training loss: 0.226521, training acc: 0.904561, valid loss: 1.462381, valid acc: 0.507624



test minibatch loop: 100%|██████████| 67/67 [00:03<00:00, 19.23it/s, accuracy=0.619, cost=1.1] 

time taken: 38.66338562965393
epoch: 5, training loss: 0.157889, training acc: 0.934693, valid loss: 1.671883, valid acc: 0.514210

break epoch:6



In [10]:
real_Y, predict_Y = [], []

pbar = tqdm(
    range(0, len(test_X), batch_size), desc = 'validation minibatch loop'
)
for i in pbar:
    batch_x = test_X[i : min(i + batch_size, test_X.shape[0])]
    batch_y = test_Y[i : min(i + batch_size, test_X.shape[0])]
    batch_memory = np.ones(([n_layer, batch_size, maxlen, d_model]))
    predict_Y += np.argmax(
        sess.run(
            model.logits, feed_dict = {model.X: batch_x, model.Y: batch_y,
                                      model.memory:batch_memory}
        ),
        1,
    ).tolist()
    real_Y += batch_y

validation minibatch loop: 100%|██████████| 67/67 [00:03<00:00, 19.05it/s]


In [11]:
print(metrics.classification_report(real_Y, predict_Y, target_names = trainset.target_names))

             precision    recall  f1-score   support

   negative       0.53      0.52      0.52      1098
   positive       0.50      0.50      0.50      1035

avg / total       0.51      0.51      0.51      2133

